Wei Wei has  [published a code][1] to normalize predictions distributions to match the estimated distribution of the test set used for public LB.  Here is a faster and more accurate code to do it.

It yields modest gains on my models, but it may make a difference on yours.  


  [1]: https://www.kaggle.com/c/two-sigma-connect-rental-listing-inquiries/discussion/31067

In [3]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

df = pd.read_json(open("../input/train.json", "r"))
df.head(2)
# Any results you write to the current directory are saved as output.

,bathrooms,bedrooms,building_id,created,description,display_address,features,interest_level,latitude,listing_id,longitude,manager_id,photos,price,street_address
10,1.5,3,53a5b119ba8f7b61d4e010512e0dfc85,2016-06-24 07:54:24,A Brand New 3 Bedroom 1.5 bath ApartmentEnjoy ...,Metropolitan Avenue,[],medium,40.7145,7211212,-73.9425,5ba989232d0489da1b5f2c45f6688adc,[https://photos.renthop.com/2/7211212_1ed4542e...,3000,792 Metropolitan Avenue
10000,1.0,2,c5c8a357cba207596b04d1afd1e4f130,2016-06-12 12:19:27,,Columbus Avenue,"[Doorman, Elevator, Fitness Center, Cats Allow...",low,40.7947,7150865,-73.9667,7533621a882f71e25173b27e3139d83d,[https://photos.renthop.com/2/7150865_be3306c5...,5465,808 Columbus Avenue


In [4]:
interest_levels = ['low', 'medium', 'high']

tau_train = {
    'low': 0.694683, 
    'medium': 0.227529,
    'high': 0.077788, 
}

tau_test = {
    'low': 0.69195995, 
    'medium': 0.23108864,
    'high': 0.07695141, 
}

def correct(df, train=True, verbose=False):
    if train:
        tau = tau_train
    else:
        tau = tau_test
        
    df_sum = df[interest_levels].sum(axis=1)
    df_correct = df[interest_levels].copy()
    
    if verbose:
        y = df_correct.mean()
        a = [tau[k] / y[k]  for k in interest_levels]
        print( a)
    
    for c in interest_levels:
        df_correct[c] /= df_sum

    for i in range(20):
        for c in interest_levels:
            df_correct[c] *= tau[c] / df_correct[c].mean()

        df_sum = df_correct.sum(axis=1)

        for c in interest_levels:
            df_correct[c] /= df_sum
    
    if verbose:
        y = df_correct.mean()
        a = [tau[k] / y[k]  for k in interest_levels]
        print( a)

    return df_correct


In [9]:
df.interest_level

10        medium
10000        low
100004      high
100007       low
100013       low
100014    medium
100016       low
100020       low
100026    medium
100027       low
100030       low
10004        low
100044      high
100048       low
10005        low
100051    medium
100052       low
100053       low
100055       low
100058       low
100062       low
100063    medium
100065       low
100066       low
10007     medium
100071       low
100075    medium
100076       low
100079      high
100081       low
           ...  
99915        low
99917        low
99919     medium
99921     medium
99923        low
99924        low
99931        low
99933        low
99935        low
99937        low
9994         low
99953        low
99956        low
99960     medium
99961        low
99964     medium
99965        low
99966        low
99979        low
99980        low
99982       high
99984        low
99986        low
99987        low
99988     medium
9999      medium
99991        low
99992        l